<a href="https://colab.research.google.com/github/JherryAlagon/Projeto-Alura/blob/main/Imers%C3%A3o_Aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -U -q google-generativeai

Importações e configurações iniciais:

In [12]:
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
GOOGLE_API_KEY= userdata.get("Secret_key")
genai.configure(api_key=GOOGLE_API_KEY)

In [13]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)


models/embedding-001
models/text-embedding-004


In [18]:
#Exemplo 1:
title = "Nunca acreditei que!"
sample_text =("que iria conseguir realizar um chatbot",
            "\n",
            "Artigo completo: \n",
            "\n",
            "Gemini API & Google")


embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [[0.016592545, -0.054461308, -0.058649313, -0.014251905, 0.064917795, -0.032206547, 0.008925886, -0.015312874, 0.009697007, 0.02043378, -0.031898238, 0.007253972, -0.044142436, -0.008267898, 0.027182406, -0.05856895, 0.0018248453, 0.055533957, 0.007677755, 0.030669274, 0.012034043, 0.024521146, -0.029262641, -0.003134055, 0.015532283, -0.01598836, -0.018651614, -0.054634623, -0.05388437, 0.012425637, -0.06962524, 0.07574544, -0.013179214, -0.041057948, -0.028488155, -0.07245296, -0.008364876, 0.05188445, 0.024071217, -0.03015622, 0.017970704, -0.075355604, -0.027717317, -0.003072601, 0.004175733, -0.028946405, -0.02522473, 0.02148608, -0.016832858, -0.09063829, 0.03848747, -0.006288985, 0.12609562, -0.010698574, -0.014593158, -0.0037834074, 0.08002426, -0.010509627, 0.01348314, 0.017877055, 0.0127154775, -0.01614796, -0.01555782, 0.04704726, 0.012955753, -0.008955, 0.016395127, 0.021696163, 0.03402467, -0.036017098, -0.010984559, -0.01787891, -0.013155677, -0.03604615, -0

In [24]:
#Listagem de documentos
DOCUMENT1 = {
    "Título" : "Operação Condor",
    "Conteúdo" : "Uma operação militar que aconteceu no Peru"}
DOCUMENT2 = {
    "Título" : "Final da Champions",
    "Conteúdo" : "Será entre o Borussia vs Real Madrid"}
DOCUMENT3 = {
    "Título" : "La U campeon de Peru",
    "Conteúdo" : "Tras derrotar 4-1 a Sporting Cristal"}


documents = [DOCUMENT1,DOCUMENT2, DOCUMENT3]

In [32]:
df = pd.DataFrame(documents)
df.columns = ["Titulo" , "Conteudo"]
df

,Titulo,Conteudo
0,Operação Condor,Uma operação militar que aconteceu no Peru
1,Final da Champions,Será entre o Borussia vs Real Madrid
2,La U campeon de Peru,Tras derrotar 4-1 a Sporting Cristal


In [34]:
model = "models/embedding-001"

In [35]:
def embed_fn(title,text,model) :
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [37]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis =1)
df

,Titulo,Conteudo,Embeddings
0,Operação Condor,Uma operação militar que aconteceu no Peru,"[-0.0030700688, -0.021113915, -0.0597755, 0.02..."
1,Final da Champions,Será entre o Borussia vs Real Madrid,"[0.012739226, -0.05153559, -0.042830236, -0.03..."
2,La U campeon de Peru,Tras derrotar 4-1 a Sporting Cristal,"[0.044717636, -0.01722302, -0.042399913, -0.04..."


In [39]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta =  genai.embed_content(model=model,
                                 consulta=consulta,
                                 task_type="RETRIEVAL_ QUERY")